In [1]:
import time  # time para el ttl
import pickle  # para guardar el objeto
from sortedcontainers import SortedDict  # para ordenar los diccionarios
import zlib  # para comprimir los valores
import os  # para crear directorios
import shutil  # para mover directorios


class LSMTree:
    def __init__(self, threshold=100, compression=True, db_path='./my_database'):
        # Inicializa un LSMTree con los parámetros especificados.
        # threshold: Umbral de tamaño de la memtable antes de que se active la escritura en disco.
        # compression: Indica si se debe comprimir el valor almacenado en disco.
        # db_path: Ruta del directorio de la base de datos.
        self.threshold = threshold
        self.compression = compression
        self.memtable = {}  # Memtable para almacenar las claves y valores temporalmente en memoria
        self.sstables = []  # Lista de SSTables (sorted string tables) que almacenan las claves y valores en disco
        self.db_path = os.path.abspath(db_path)  # Ruta absoluta del directorio de la base de datos
        self.index = {}  # Índice para mapear las claves con su ubicación en las SSTables
        os.makedirs(self.db_path, exist_ok=True)  # Crear el directorio si no existe


    def put(self, key, value, ttl=None):
        # Inserta una clave y su valor asociado en el LSMTree.
        # key: Clave a insertar.
        # value: Valor asociado a la clave.
        # ttl: Tiempo de vida (opcional) de la clave en segundos.
        self.memtable[key] = {'value': value, 'timestamp': time.time(), 'ttl': ttl}

        if len(self.memtable) >= self.threshold:
            self.flush_memtable()

    def get(self, key):
        # Obtiene el valor asociado a una clave en el LSMTree.
        # key: Clave a buscar.
        # return: Valor asociado a la clave si existe, None en caso contrario.
        if key in self.memtable:
            entry = self.memtable[key]
            if self.is_entry_valid(entry):
                return entry['value']
            else:
                del self.memtable[key]

        if key in self.index:
            sstable_index, sstable_key = self.index[key]
            sstable = self.sstables[sstable_index]
            entry = sstable[sstable_key]
            if self.is_entry_valid(entry):
                return entry['value']
            else:
                del sstable[sstable_key]
                del self.index[key]

        return None

    def is_entry_valid(self, entry):
        # Comprueba si una entrada es válida basándose en su tiempo de vida (ttl).
        # entry: Entrada a comprobar.
        # return: True si la entrada es válida, False si ha expirado.
        if entry['ttl'] is not None and (time.time() - entry['timestamp']) >= entry['ttl']:
            return False
        return True

    def flush_memtable(self):
        # Transfiere los datos de la memtable a las SSTables y realiza la fusión si es necesario.
        self.sstables.append(self.memtable)
        self.index_memtable()

        self.memtable = {}

        if self.compression:
            self.compress_sstable(self.sstables[-1])

        self.merge_sstables()

    def compress_sstable(self, sstable):
        # Comprime los valores almacenados en una SSTable.
        # sstable: SSTable a comprimir.
        for key, entry in sstable.items():
            compressed_value = zlib.compress(entry['value'].encode())
            entry['value'] = compressed_value

    def merge_sstables(self):
        # Fusiona las SSTables si hay más de una en la lista de SSTables.
        if len(self.sstables) > 1:
            merged_sstable = {}

            for sstable in self.sstables:
                for key, entry in sstable.items():
                    merged_sstable[key] = entry

            self.sstables = [merged_sstable]

    def index_memtable(self):
        # Indexa las claves de la memtable para mantener el mapeo con las SSTables.
        for key in self.memtable:
            self.index[key] = (len(self.sstables), key)

    def persist_sstables(self):
        # Guarda las SSTables en disco y actualiza la ubicación de la base de datos.
        new_db_path = f'{self.db_path}_new'

        if not os.path.exists(new_db_path):
            os.makedirs(new_db_path)

        for i, sstable in enumerate(self.sstables):
            filename = f'{new_db_path}/sstable_{i}.txt'

            with open(filename, 'w') as file:
                for key, entry in sstable.items():
                    file.write(f'{key},{entry["value"]},{entry["timestamp"]},{entry["ttl"]}\n')

        shutil.rmtree(self.db_path)
        shutil.move(new_db_path, self.db_path)

        self.sstables = []

    def load_sstables(self):
        # Carga las SSTables existentes en disco en la memoria.
        if not os.path.exists(self.db_path):
            return

        self.sstables = []

        for filename in os.listdir(self.db_path):
            sstable = {}

            with open(f'{self.db_path}/{filename}', 'r') as file:
                for line in file:
                    parts = line.strip().split(',')
                    key = parts[0]
                    value = parts[1]
                    timestamp = float(parts[2])
                    ttl = float(parts[3])

                    sstable[key] = {'value': value, 'timestamp': timestamp, 'ttl': ttl}

            self.sstables.append(sstable)

        self.build_index()

    def build_index(self):
        # Construye el índice para mapear las claves con su ubicación en las SSTables.
        self.index = {}

        for i, sstable in enumerate(self.sstables):
            for key in sstable:
                self.index[key] = (i, key)

    def save_to_disk(self, filepath):
        # Guarda el LSMTree en disco utilizando pickle.
        # filepath: Ruta del archivo en el que se guardará el LSMTree.
        with open(filepath, 'wb') as file:
            pickle.dump(self, file)

    @staticmethod
    def load_from_disk(filepath):
        # Carga un LSMTree desde un archivo utilizando pickle.
        # filepath: Ruta del archivo desde el que se cargará el LSMTree.
        with open(filepath, 'rb') as file:
            return pickle.load(file)

    def close(self):
        # Guardar las SSTables en disco y limpiar las variables del LSMTree
        self.persist_sstables()
        self.memtable = {}
        self.sstables = []
        self.index = {}


In [2]:
# Crear un LSMTree
lsm_tree = LSMTree()
filepath = './my_database/lsm_tree.pkl'

# Insertar claves y valores en el LSMTree
lsm_tree.put("key1", "value1")
lsm_tree.put("key2", "value2")
lsm_tree.put("key3", "value3")
lsm_tree.put("key4", "value4")

# Obtener valores de claves
value1 = lsm_tree.get("key1")  # Retorna "value1"
value2 = lsm_tree.get("key2")  # Retorna "value2"
value3 = lsm_tree.get("key3")  # Retorna "value3"
value4 = lsm_tree.get("key4")  # Retorna "value4"
value5 = lsm_tree.get("key5")  # Retorna None, ya que la clave no existe

# Imprimir los valores obtenidos
print(value1)  # Imprime "value1"
print(value2)  # Imprime "value2"
print(value3)  # Imprime "value3"
print(value4)  # Imprime "value4"
print(value5)  # Imprime None

# Actualizar un valor existente
lsm_tree.put("key2", "new_value2")
updated_value2 = lsm_tree.get("key2")  # Retorna "new_value2"
print(updated_value2)  # Imprime "new_value2"

# Insertar claves temporales con tiempo de vida
lsm_tree.put("temp_key1", "temp_value1", ttl=4)  # Clave temporal con tiempo de vida de 4 segundos
lsm_tree.put("temp_key2", "temp_value2", ttl=5)  # Clave temporal con tiempo de vida de 5 segundos

time.sleep(6)  # Esperar 15 segundos para que las claves temporales expiren

# Obtener valores de claves temporales expiradas
expired_value1 = lsm_tree.get("temp_key1")  # Retorna None, ya que la clave ha expirado
expired_value2 = lsm_tree.get("temp_key2")  # Retorna None, ya que la clave ha expirado
print(expired_value1)  # Imprime None
print(expired_value2)  # Imprime None

# Guardar el LSMTree en disco
lsm_tree.persist_sstables()
lsm_tree.save_to_disk(filepath)

# Cerrar el LSMTree
# lsm_tree.close()

# Cargar el LSMTree desde el archivo en disco
new_lsm_tree = LSMTree.load_from_disk(filepath)

# Obtener valores de claves desde el LSMTree cargado
loaded_value1 = new_lsm_tree.get("key1")  # Retorna "value1"
loaded_value2 = new_lsm_tree.get("key2")  # Retorna "new_value2"
print(loaded_value1)  # Imprime "value1"
print(loaded_value2)  # Imprime "new_value2"

# Cerrar el LSMTree cargado
#new_lsm_tree.close()a

value1
value2
value3
value4
None
new_value2
None
None
value1
new_value2


1. `__init__(self, threshold=100, compression=True, db_path='./my_database')`: Es el método de inicialización de la clase. Se encarga de configurar los parámetros iniciales del LSMTree, como el umbral de tamaño de la memtable, la opción de compresión y la ruta del directorio de la base de datos.

2. `put(self, key, value, ttl=None)`: Inserta una clave y su valor asociado en el LSMTree. Recibe la clave (`key`), el valor (`value`) y un tiempo de vida opcional (`ttl`) para la clave en segundos. Guarda la entrada en la memtable y verifica si se ha alcanzado el umbral de tamaño para realizar la escritura en disco.

3. `get(self, key)`: Obtiene el valor asociado a una clave en el LSMTree. Recibe la clave (`key`) y devuelve el valor asociado si existe. Busca primero en la memtable y luego en las SSTables utilizando el índice. Verifica también si la entrada ha expirado según su tiempo de vida.

4. `is_entry_valid(self, entry)`: Comprueba si una entrada es válida basándose en su tiempo de vida (`ttl`). Recibe la entrada (`entry`) y devuelve `True` si la entrada es válida o `False` si ha expirado.

5. `flush_memtable(self)`: Transfiere los datos de la memtable a las SSTables y realiza la fusión si es necesario. Agrega la memtable actual a la lista de SSTables, indexa las claves de la memtable, comprime los valores si está habilitada la compresión y fusiona las SSTables si hay más de una.

6. `compress_sstable(self, sstable)`: Comprime los valores almacenados en una SSTable. Recibe la SSTable (`sstable`) y comprime los valores utilizando el algoritmo de compresión zlib.

7. `merge_sstables(self)`: Fusiona las SSTables si hay más de una en la lista de SSTables. Crea una nueva SSTable fusionando todas las entradas de las SSTables existentes.

8. `index_memtable(self)`: Indexa las claves de la memtable para mantener el mapeo con las SSTables. Crea entradas en el índice para cada clave en la memtable, indicando su ubicación en la lista de SSTables.

9. `persist_sstables(self)`: Guarda las SSTables en disco y actualiza la ubicación de la base de datos. Crea un nuevo directorio para las SSTables, guarda cada SSTable en un archivo separado y elimina el directorio anterior.

10. `load_sstables(self)`: Carga las SSTables existentes en disco en la memoria. Lee los archivos de las SSTables, crea las SSTables correspondientes en la memoria y construye el índice.

11. `build_index(self)`: Construye el índice para mapear las claves con su ubicación en las SSTables. Recorre las SSTables y crea entradas en el índice para cada clave en cada SSTable.

12. `save_to_disk(self, filepath)`: Guarda el LSMTree en disco utilizando el módulo `pickle`. Recibe la ruta del archivo (`filepath`) en el que se guardará el LSMTree y lo guarda en formato binario.

13. `load_from_disk(filepath)`: C

arga un LSMTree desde un archivo utilizando el módulo `pickle`. Recibe la ruta del archivo (`filepath`) desde el que se cargará el LSMTree y lo carga desde el archivo binario.

14. `close(self)`: Guarda las SSTables en disco, limpia las variables del LSMTree y cierra el LSMTree. Llama a `persist_sstables()` para guardar las SSTables en disco, y luego limpia las variables relacionadas con la memtable, las SSTables y el índice.